In [ ]:
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import os

def G_test_vis(encoder_model, dataloader, save_dir='./'):
    encoder_model.eval()
    x = []
    y = []
    
    for data in dataloader:
        data = data.to('cuda')
        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)
        
        g1, g2 = encoder_model(data.x, data.edge_index, data.batch)
        x.append(g1 + g2)
        y.append(data.y)

    x = torch.cat(x, dim=0)
    y = torch.cat(y, dim=0)

    x_np = x.detach().cpu().numpy() 
    y_np = y.detach().cpu().numpy()  

    tsne = TSNE(n_components=2, random_state=42)
    x_embedded = tsne.fit_transform(x_np) 

    plt.figure(figsize=(10, 8))

    unique_labels = np.unique(y_np)
    colors=['#6a3d9a','#33a02c','#46A3FF'] 
    # colors = ['#6a3d9a','#46A3FF','#33a02c']  
    for i, label in enumerate(unique_labels):
        idx = y_np == label
        count = np.sum(idx)
        plt.scatter(x_embedded[idx, 0], x_embedded[idx, 1], s=5, label=f'Class {label} ({count} points)', color=colors[i % len(colors)])  

    plt.title('t-SNE visualization of graph embeddings', fontsize=20)
    plt.xlabel('t-SNE component 1', fontsize=20)
    plt.ylabel('t-SNE component 2', fontsize=20)

    plt.legend(fontsize=15)
    # ax = plt.gca()  
    # ax.spines['top'].set_visible(False)    
    # ax.spines['right'].set_visible(False)  
    # ax.spines['bottom'].set_visible(False) 
    # ax.spines['left'].set_visible(False)   
    # ax.set_xticks([])  
    # ax.set_yticks([])  
    # ax.tick_params(left=False, bottom=False)  

    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(os.path.join(save_dir, 'tsne_visualization.pdf'))
        print(f't-SNE visualization saved to {os.path.join(save_dir, "tsne_visualization.pdf")}')

    plt.show()

In [ ]:
def G_test_3d_vis(encoder_model, dataloader, save_dir='./'):
    encoder_model.eval()
    x = []
    y = []
    
    for data in dataloader:
        data = data.to('cuda')
        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)
        
        g1, g2 = encoder_model(data.x, data.edge_index, data.batch)
        x.append(g1 + g2)
        y.append(data.y)

    x = torch.cat(x, dim=0)
    y = torch.cat(y, dim=0)

    x_np = x.detach().cpu().numpy()  
    y_np = y.detach().cpu().numpy()  
    tsne = TSNE(n_components=3, random_state=42)  
    x_embedded = tsne.fit_transform(x_np) 
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d') 

    unique_labels = np.unique(y_np)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))  
    for label, color in zip(unique_labels, colors):
        idx = y_np == label
        count = np.sum(idx)
        ax.scatter(x_embedded[idx, 0], x_embedded[idx, 1], x_embedded[idx, 2], 
                   c=[color], s=2, label=f'Class {label} ({count} points)')
    
    ax.set_title('3D t-SNE visualization of graph embeddings')
    ax.set_xlabel('t-SNE component 1')
    ax.set_ylabel('t-SNE component 2')
    ax.set_zlabel('t-SNE component 3')
    ax.legend()  

    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(os.path.join(save_dir, 'tsne_visualization_3d.pdf'))
        print(f'3D t-SNE visualization saved to {os.path.join(save_dir, "tsne_visualization_3d.pdf")}')

    plt.show()  

    return 0

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio
from sklearn.manifold import TSNE
import numpy as np
import os
import torch
def plot_interactive_3d_tsne(x_embedded, y_np, save_dir='./'):
    unique_labels = np.unique(y_np)
    colors = [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})' 
              for r, g, b in plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))[:, :3]]

    traces = []
    for label, color in zip(unique_labels, colors):
        idx = y_np == label
        trace = go.Scatter3d(
            x=x_embedded[idx, 0],
            y=x_embedded[idx, 1],
            z=x_embedded[idx, 2],
            mode='markers',
            name=f'Class {label}',
            marker=dict(
                size=1,
                color=color,
                opacity=0.8
            ),
            text=[f'Class: {label}' for _ in range(np.sum(idx))],  # 悬停文本
            hoverinfo='text'
        )
        traces.append(trace)

    layout = go.Layout(
        title='Interactive 3D t-SNE visualization of graph embeddings',
        scene=dict(
            xaxis_title='t-SNE component 1',
            yaxis_title='t-SNE component 2',
            zaxis_title='t-SNE component 3',
        ),
        legend_title_text='Classes'
    )

    fig = go.Figure(data=traces, layout=layout)

    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)
        pio.write_html(fig, file=os.path.join(save_dir, 'interactive_tsne_3d.html'))
        print(f'Interactive 3D t-SNE visualization saved to {os.path.join(save_dir, "interactive_tsne_3d.html")}')

    fig.show()

def G_test_3d_interactive_vis(encoder_model, dataloader, save_dir='./'):
    encoder_model.eval()
    x = []
    y = []
    
    for data in dataloader:
        data = data.to('cuda')
        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)
        
        g1, g2 = encoder_model(data.x, data.edge_index, data.batch)
        x.append(g1 + g2)
        y.append(data.y)

    x = torch.cat(x, dim=0)
    y = torch.cat(y, dim=0)

    x_np = x.detach().cpu().numpy()
    y_np = y.detach().cpu().numpy()

    tsne = TSNE(n_components=3, random_state=42)
    x_embedded = tsne.fit_transform(x_np)

    plot_interactive_3d_tsne(x_embedded, y_np, save_dir)

    return 0

In [ ]:
##radar chart
import numpy as np
import matplotlib.pyplot as plt

# Data
models = ['Model1', 'Model2', 'Model3', 'Model4', 'Model5']
datasets = ['Metric1', 'Metric2', 'Metric3', 'Metric4', 'Metric5', 'Metric6']
values = np.array([
    [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
    [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
    [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
    [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
    [1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
])

# Set angles for the radar chart
angles = np.linspace(0, 2*np.pi, len(datasets), endpoint=False)

# Close the plot
values = np.c_[values, values[:, 0]]
angles = np.concatenate((angles, [angles[0]]))

# Set figure size and style
plt.figure(figsize=(12, 10))
ax = plt.subplot(111, projection='polar')

# Use a new color scheme, Model5 in blue
colors = ['#e15759', '#f28e2b', '#76b7b2', '#59a14f', '#1f77b4']
markers = ['o', 's', '^', 'D', 'v']

# Set marker size
marker_size = 10  # You can adjust this value to change the marker size

# Plot data for each model
for i, model in enumerate(models):
    ax.plot(angles, values[i], marker=markers[i], linewidth=2.5, label=model, color=colors[i], markersize=marker_size)
    ax.fill(angles, values[i], alpha=0.1, color=colors[i])

# Set tick labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(datasets, fontsize=20, fontweight='bold')

# Add legend
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1), fontsize=20)

# Set y-axis range and labels, starting from 50
ax.set_ylim(50, 100)
ax.set_yticks(np.arange(50, 101, 10))
ax.set_yticklabels([f'{i}%' for i in range(50, 101, 10)], fontsize=18)

# Add grid lines
ax.grid(True, color='gray', alpha=0.3)

# Set tick line color
ax.tick_params(axis='both', colors='black')

# Set title
plt.title('Model Performance Comparison', fontsize=20, fontweight='bold', y=1.08)

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig('radar_chart_final.pdf', dpi=300, bbox_inches='tight')

# Display the plot
plt.show()